# General Stuff

---

In [32]:
import os
import geopandas as gpd
import pandas as pd

import numpy as np
import math
from math import sin, cos, radians, pi

from pyproj import Geod

In [2]:
SCRIPT_PATH = "/Users/Syrin/Documents/GitHub/AI4PublicHealth/"

# Read the road points w/ elevation .shp file via geopandas and store as a pandas dataframe.
ROAD_POINT_WITH_ELEVATION_PATH = os.path.join(SCRIPT_PATH, "Datasets/MZUZU_roads_pointdata_with_elevation.shp")
ROAD_POINT_WITH_ELEVATION_DATA = gpd.read_file(ROAD_POINT_WITH_ELEVATION_PATH)
ROAD_POINT_WITH_ELEVATION_DATA = pd.DataFrame(ROAD_POINT_WITH_ELEVATION_DATA)

# Create DataFrame

---

In [39]:
test = 2

radius_earth = 6378.137

def road_elevation_processing(road_elevation_df):
    """Clean the .shp file that contains the route data. Create a second pandas data frame to store a processed
        version of the original data from the .shp file. """

    # Create a secondary pandas data frame that contains the index of nodes, start/end longitude and latitude,
    # elevation, road condition, and road type.
    processed_data = []

    
    for rows in range(len(road_elevation_df.index)):
        
        # Get the coordinates.
        coordinates = list(road_elevation_df.iloc[rows, 22].coords)
        start_latitude = coordinates[0][1] # Contains max 7 decimal points
        start_longitude = coordinates[0][0] # Contains max 7 decimal points

        # Get the elevation, road condition, and road type.
        elevation = road_elevation_df.iloc[rows, 17]
        road_condition = road_elevation_df.iloc[rows, 10]
        road_type = road_elevation_df.iloc[rows, 9]
        
        # Convert distance and angle to connecting node coords. Note the rounding is to 7 decimal points.
        distance = road_elevation_df.iloc[rows, 15] # Contains max 14 decimal points
        angle = road_elevation_df.iloc[rows, 16]# Contains max 14 decimal points
            
        if test == 1:
            radians = math.radians(angle) # Contains max 16 decimal points


            # Convert the start_longitude and latitude to radians.
            from_long = math.radians(start_longitude) # Contains max 16 data points
            from_lat = math.radians(start_latitude)# Contains 17 data points

            # print(len(str(distance)), len(str(angle)), len(str(radians)), len(str(from_long)), len(str(from_lat)))

            # to_lat has 17 decimal points. to_long has 16 decimal points
            to_lat = math.asin(sin(from_lat) * 
                               cos(distance / radius_earth) + 
                               cos(from_lat) * 
                               sin(distance / radius_earth) * 
                               math.cos(radians))
            to_long = from_long + math.atan2(sin(radians) * sin(distance / radius_earth) * cos(from_lat), 
                                             cos(distance / radius_earth) - sin(from_lat) * sin(to_lat))

            to_lat = math.degrees(to_lat) # + 0.0000428
            to_long = math.degrees(to_long) # + (-0.0000254)
    
        else:
            geod = Geod(ellps="WGS84")
            
            to_long, to_lat, _ = geod.fwd(start_longitude, start_latitude, angle, distance, radians = False)
        
        start_longitude = start_longitude
        start_latitude = start_latitude
        to_lat = round((to_lat), 7)
        to_long = round(to_long, 7)

        processed_data.append((start_longitude, start_latitude, elevation, distance, angle, to_long, to_lat, road_condition, road_type, [], []))

    processed_data = pd.DataFrame(processed_data)

    processed_data = processed_data.rename(
        columns={0: "Longitude", 1: "Latitude", 2: "Elevation", 3: "Distance", 4: "Angle", 5: "To Longitude", 6: "To Latitude", 7: "Road Condition", 8: "Road Type", 9: "Connection(s)"})

    return processed_data


road_elevation_nodes = road_elevation_processing(ROAD_POINT_WITH_ELEVATION_DATA)

road_elevation_nodes

Longitude   Latitude  Elevation  Distance       Angle  To Longitude  \
0      34.029856 -11.458530       1286  0.000623  143.665285     34.029856   
1      34.028073 -11.458322       1279  0.000674   68.613613     34.028073   
2      34.028159 -11.458254       1279  0.000784   68.912626     34.028159   
3      34.028325 -11.458243       1277  0.000950   92.006825     34.028325   
4      34.028443 -11.458259       1277  0.001069  116.018906     34.028443   
...          ...        ...        ...       ...         ...           ...   
58127  34.034627 -11.458959       1270  0.002183   78.742212     34.034627   
58128  34.034729 -11.458953       1270  0.002285   94.031740     34.034729   
58129  34.034887 -11.458986       1275  0.002447  100.120003     34.034887   
58130  34.029513 -11.458301       1282  0.000201  102.616360     34.029513   
58131  34.029681 -11.458360       1283  0.000379  121.730663     34.029681   

       To Latitude Road Condition Road Type Connection(s)  10  
0       -11.458530           None      path            []  []  
1       -11.458322           None     track            []  []  
2       -11.458254           None     track            []  []  
3       -11.458243           None     track            []  []  
4       -11.458259           None     track            []  []  
...            ...            ...       ...           ...  ..  
58127   -11.458959           None      path            []  []  
58128   -11.458953           None      path            []  []  
58129   -11.458986           None      path            []  []  
58130   -11.458301           None      path            []  []  
58131   -11.458360           None      path            []  []  

[58132 rows x 11 columns]

# Check that the math is correct

---

In [28]:
lat1 = road_elevation_nodes.loc[37757, "Latitude"]
long1 = road_elevation_nodes.loc[37757, "Longitude"]
lat2 = road_elevation_nodes.loc[37758, "To Latitude"]
long2 = road_elevation_nodes.loc[37758, "To Longitude"]

print(lat1, long1)
print(lat2, long2)
print("")
lat_diff = lat1 - lat2
long_diff = long1 - long2
print("Lat diff", f'{lat_diff:.7f}')
print("Long diff", f'{long_diff:.7f}')

-11.4643784 33.9784974
-11.4644212 33.9785228

Lat diff 0.0000428
Long diff -0.0000254


In [40]:
lat1 = road_elevation_nodes.loc[37757, "Latitude"]
long1 = road_elevation_nodes.loc[37757, "Longitude"]
lat2 = road_elevation_nodes.loc[37758, "To Latitude"]
long2 = road_elevation_nodes.loc[37758, "To Longitude"]

print(lat1, long1)
print(lat2, long2)
print("")
lat_diff = lat1 - lat2
long_diff = long1 - long2
print("Lat diff", f'{lat_diff:.7f}')
print("Long diff", f'{long_diff:.7f}')

-11.4643784 33.9784974
-11.4644109 33.9785172

Lat diff 0.0000325
Long diff -0.0000198


In [26]:
road_elevation_long_list = road_elevation_nodes["Longitude"].values
print(road_elevation_long_list)

road_elevation_lat_list = road_elevation_nodes["Latitude"].values
print(road_elevation_lat_list)

if 34.029860 in road_elevation_long_list:
    print("Is here")
if -11.4585340 in road_elevation_lat_list:
    print("Is here too")

[34.0298563 34.0280727 34.0281585 ... 34.0348868 34.029513  34.0296807]
[-11.4585297 -11.458322  -11.4582536 ... -11.4589857 -11.4583009
 -11.4583602]
Is here


In [27]:
long_unique_arr = road_elevation_nodes["Longitude"].unique()
lat_unique_arr = road_elevation_nodes["Latitude"].unique()

print(len(long_unique_arr))
print(len(lat_unique_arr))

52612
53774


# Create Dict

---

In [17]:
road_elevation_dict = {}

for index, row, in road_elevation_nodes.iterrows():
    lat_long_pair = (round(row["To Latitude"], 6), round(row["To Longitude"], 6))
    if lat_long_pair not in road_elevation_dict:
        road_elevation_dict[lat_long_pair] = [index]
    else: 
        road_elevation_dict[lat_long_pair].append(index)

road_elevation_dict

{(-11.458534, 34.02986): [0],
 (-11.45832, 34.028078): [1],
 (-11.458251, 34.028165): [2],
 (-11.458243, 34.028334): [3],
 (-11.458263, 34.028452): [4],
 (-11.45832, 34.028512): [5],
 (-11.4583, 34.028669): [6],
 (-11.4583, 34.028772): [7],
 (-11.458293, 34.028842): [8],
 (-11.458256, 34.028945): [9],
 (-11.458255, 34.029065): [10],
 (-11.458276, 34.0292): [11],
 (-11.459063, 34.032584): [12],
 (-11.459101, 34.03296): [13],
 (-11.459154, 34.033226): [14],
 (-11.459251, 34.033545): [15],
 (-11.457493, 34.045662): [16],
 (-11.457404, 34.045806): [17],
 (-11.457358, 34.045923): [18],
 (-11.45731, 34.046061): [19],
 (-11.457294, 34.046167): [20],
 (-11.457311, 34.046259): [21],
 (-11.457328, 34.046321): [22],
 (-11.457325, 34.046365): [23],
 (-11.457323, 34.046408): [24],
 (-11.457309, 34.046446): [25],
 (-11.457317, 34.046473): [26],
 (-11.457342, 34.046506): [27],
 (-11.457364, 34.046551): [28],
 (-11.458353, 34.027657): [29],
 (-11.458338, 34.027862): [30],
 (-11.459781, 34.03818): [31]

In [18]:
for index, row in road_elevation_nodes.iterrows():
    lat_long_pair = (round(row["Latitude"], 6), round(row["Longitude"], 6))
    
    if lat_long_pair in road_elevation_dict:
        fill_dataframe = road_elevation_dict[lat_long_pair]
        print(fill_dataframe[0])
        road_elevation_nodes.iloc[index, 9] = fill_dataframe[0]

# More Testing

---

In [19]:
for index, row in road_elevation_nodes.iterrows(): 
    if row["Distance"] == 0.00129511125876:
        print(index)
    
for index, row in road_elevation_nodes.iterrows(): 
    if row["Distance"] == 0.00125705485326:
        print(index)

37758
37757


In [20]:
road_elevation_nodes.iloc[37758]

Longitude           33.978517
Latitude            33.978517
Elevation                1316
Distance             0.001295
Angle              151.843585
To Longitude        33.978523
To Latitude        -11.464421
Road Condition           None
Road Type         residential
Connection(s)              []
10                         []
Name: 37758, dtype: object

In [21]:
road_elevation_nodes.iloc[37757]

Longitude           33.978497
Latitude            33.978497
Elevation                1316
Distance             0.001257
Angle              145.569882
To Longitude        33.978504
To Latitude        -11.464388
Road Condition           None
Road Type         residential
Connection(s)              []
10                         []
Name: 37757, dtype: object